# Assignment 2: Advanced RAG Techniques
## Day 6 Session 2 - Advanced RAG Fundamentals

**OBJECTIVE:** Implement advanced RAG techniques including postprocessors, response synthesizers, and structured outputs.

**LEARNING GOALS:**
- Understand and implement node postprocessors for filtering and reranking
- Learn different response synthesis strategies (TreeSummarize, Refine)
- Create structured outputs using Pydantic models
- Build advanced retrieval pipelines with multiple processing stages

**DATASET:** Use the same data folder as Assignment 1 (`Day_6/session_2/data/`)

**PREREQUISITES:** Complete Assignment 1 first

**INSTRUCTIONS:**
1. Complete each function by replacing the TODO comments with actual implementation
2. Run each cell after completing the function to test it
3. The answers can be found in the `03_advanced_rag_techniques.ipynb` notebook
4. Each technique builds on the previous one


In [3]:
# Import required libraries for advanced RAG
import os
from pathlib import Path
from typing import Dict, List, Optional, Any
from pydantic import BaseModel, Field

from dotenv import load_dotenv

# Core LlamaIndex components
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, Settings
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever

# Vector store
from llama_index.vector_stores.lancedb import LanceDBVectorStore

# Embeddings and LLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openrouter import OpenRouter

# Advanced RAG components (we'll use these in the assignments)
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.response_synthesizers import TreeSummarize, Refine, CompactAndRefine
from llama_index.core.output_parsers import PydanticOutputParser

print("✅ Advanced RAG libraries imported successfully!")


✅ Advanced RAG libraries imported successfully!


In [4]:
# Configure Advanced RAG Settings (Using OpenRouter)
def setup_advanced_rag_settings():
    """
    Configure LlamaIndex with optimized settings for advanced RAG.
    Uses local embeddings and OpenRouter for LLM operations.
    """
    # Load environment variables from .env file
    load_dotenv()
    
    # Check for OpenRouter API key
    api_key = os.getenv("OPENROUTER_API_KEY")
    if not api_key:
        print("⚠️  OPENROUTER_API_KEY not found - LLM operations will be limited")
        print("   You can still complete postprocessor and retrieval exercises")
    else:
        print("✅ OPENROUTER_API_KEY found - full advanced RAG functionality available")
        
        # Configure OpenRouter LLM
        Settings.llm = OpenRouter(
            api_key=api_key,
            model="gpt-4o",
            temperature=0.1  # Lower temperature for more consistent responses
        )
    
    # Configure local embeddings (no API key required)
    Settings.embed_model = HuggingFaceEmbedding(
        model_name="BAAI/bge-small-en-v1.5",
        trust_remote_code=True
    )
    
    # Advanced RAG configuration
    Settings.chunk_size = 512  # Smaller chunks for better precision
    Settings.chunk_overlap = 50
    
    print("✅ Advanced RAG settings configured")
    print("   - Chunk size: 512 (optimized for precision)")
    print("   - Using local embeddings for cost efficiency")
    print("   - OpenRouter LLM ready for response synthesis")

# Setup the configuration
setup_advanced_rag_settings()


✅ OPENROUTER_API_KEY found - full advanced RAG functionality available


Loading weights: 100%|██████████| 199/199 [00:00<00:00, 1885.33it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: BAAI/bge-small-en-v1.5
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Advanced RAG settings configured
   - Chunk size: 512 (optimized for precision)
   - Using local embeddings for cost efficiency
   - OpenRouter LLM ready for response synthesis


In [5]:
# Setup: Create index from Assignment 1 (reuse the basic functionality)
def setup_basic_index(data_folder: str = "../data", force_rebuild: bool = False):
    """
    Create a basic vector index that we'll enhance with advanced techniques.
    This reuses the concepts from Assignment 1.
    """
    # Create vector store
    vector_store = LanceDBVectorStore(
        uri="./advanced_rag_vectordb",
        table_name="documents"
    )
    
    # Load documents
    if not Path(data_folder).exists():
        print(f"❌ Data folder not found: {data_folder}")
        return None
        
    reader = SimpleDirectoryReader(input_dir=data_folder, recursive=True)
    documents = reader.load_data()
    
    # Create storage context and index
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents(
        documents, 
        storage_context=storage_context,
        show_progress=True
    )
    
    print(f"✅ Basic index created with {len(documents)} documents")
    print("   Ready for advanced RAG techniques!")
    return index

# Create the basic index
print("📁 Setting up basic index for advanced RAG...")
index = setup_basic_index()

if index:
    print("🚀 Ready to implement advanced RAG techniques!")
else:
    print("❌ Failed to create index - check data folder path")


Table documents doesn't exist yet. Please add some data to create it.


📁 Setting up basic index for advanced RAG...


/Users/chandru/python-sandbox/.venv/lib/python3.12/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/Users/chandru/python-sandbox/.venv/lib/python3.12/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/Users/chandru/python-sandbox/.venv/lib/python3.12/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Generating embeddings: 100%|██████████| 94/94 [00:02<00:00, 33.04it/s]
2026-02-08 02:27:04,285 - INFO - Create new table documents adding data.


✅ Basic index created with 42 documents
   Ready for advanced RAG techniques!
🚀 Ready to implement advanced RAG techniques!


[2026-02-08T07:27:04Z WARN  lance::dataset::write::insert] No existing dataset at /Users/chandru/python-sandbox/Outskill_AI_Acc_Assignments/RAG_Atlas_session_3/assignments/advanced_rag_vectordb/documents.lance, it will be created


## 1. Node Postprocessors - Similarity Filtering

**Concept:** Postprocessors refine retrieval results after the initial vector search. The `SimilarityPostprocessor` filters out chunks that fall below a relevance threshold.

**Why it matters:** Raw vector search often returns some irrelevant results. Filtering improves precision and response quality.

Complete the function below to create a query engine with similarity filtering.


In [7]:
def create_query_engine_with_similarity_filter(index, similarity_cutoff: float = 0.3, top_k: int = 10):
    """
    Create a query engine that filters results based on similarity scores.
    
    TODO: Complete this function to create a query engine with similarity postprocessing.
    HINT: Use index.as_query_engine() with node_postprocessors parameter containing SimilarityPostprocessor
    
    Args:
        index: Vector index to query
        similarity_cutoff: Minimum similarity score (0.0 to 1.0)
        top_k: Number of initial results to retrieve before filtering
        
    Returns:
        Query engine with similarity filtering
    """
    # Create similarity postprocessor with the cutoff threshold
    similarity_processor = SimilarityPostprocessor(similarity_cutoff=similarity_cutoff)
    
    # Create query engine with similarity filtering
    query_engine = index.as_query_engine(similarity_top_k=top_k,node_postprocessors=[similarity_processor])
    
    # return query_engine
    
    # PLACEHOLDER - Replace with actual implementation
    print(f"Created query engine with similarity cutoff {similarity_cutoff}")
    return query_engine

# Test the function
if index:
    filtered_engine = create_query_engine_with_similarity_filter(index, similarity_cutoff=0.3)
    
    if filtered_engine:
        print("✅ Query engine with similarity filtering created")
        
        # Test query
        test_query = "What are the benefits of AI agents?"
        print(f"\n🔍 Testing query: '{test_query}'")
        
        # Uncomment when implemented:
        response = filtered_engine.query(test_query)
        print(f"📝 Response: {response}")
        print("   (Function is now fully implemented)")
    else:
        print("❌ Failed to create filtered query engine")
else:
    print("❌ No index available - run previous cells first")


2026-02-08 02:34:45,865 - INFO - query_type :, vector


Created query engine with similarity cutoff 0.3
✅ Query engine with similarity filtering created

🔍 Testing query: 'What are the benefits of AI agents?'


2026-02-08 02:34:47,031 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-08 02:34:48,366 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-08 02:34:50,605 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


📝 Response: AI agents offer significant benefits, including the ability to achieve complex objectives through sophisticated reasoning, planning, and tool utilization. They can autonomously engage with complex environments, make informed decisions, and assist humans in a wide range of tasks. These agents work iteratively, allowing for task breakdown, continuous improvement, and clear role allocation, which boosts their performance. Additionally, multi-agent systems can distribute tasks across multiple agents and provide feedback from various viewpoints, enhancing the overall efficiency and success of task execution.
   (Function is now fully implemented)


## 2. Response Synthesizers - TreeSummarize

**Concept:** Response synthesizers control how retrieved information becomes final answers. `TreeSummarize` builds responses hierarchically, ideal for complex analytical questions.

**Why it matters:** Different synthesis strategies work better for different query types. TreeSummarize excels at comprehensive analysis and long-form responses.

Complete the function below to create a query engine with TreeSummarize response synthesis.


In [12]:
from llama_index.core import get_response_synthesizer

def create_query_engine_with_tree_summarize(index, top_k: int = 5):
    """
    Create a query engine that uses TreeSummarize for comprehensive responses.
    
    TODO: Complete this function to create a query engine with TreeSummarize synthesis.
    HINT: Create a TreeSummarize instance, then use index.as_query_engine() with response_synthesizer parameter
    
    Args:
        index: Vector index to query
        top_k: Number of results to retrieve
        
    Returns:
        Query engine with TreeSummarize synthesis
    """
    # Create TreeSummarize response synthesizer
    tree_synthesizer = get_response_synthesizer(response_mode="tree_summarize", use_async=True)
    
    # Create query engine with the synthesizer
    query_engine = index.as_query_engine(
        similarity_top_k=top_k,
        response_synthesizer=tree_synthesizer
    )
    
    # PLACEHOLDER - Replace with actual implementation
    print(f"Created query engine with TreeSummarize synthesis")
    return query_engine

# Test the function
if index:
    tree_engine = create_query_engine_with_tree_summarize(index)
    
    if tree_engine:
        print("✅ Query engine with TreeSummarize created")
        
        # Test with a complex analytical query
        analytical_query = "Compare the advantages and disadvantages of different AI agent frameworks"
        print(f"\n🔍 Testing analytical query: '{analytical_query}'")
        
        # Uncomment when implemented:
        response = tree_engine.query(analytical_query)
        print(f"📝 TreeSummarize Response:\n{response}")
    else:
        print("❌ Failed to create TreeSummarize query engine")
else:
    print("❌ No index available - run previous cells first")


2026-02-08 02:51:16,000 - INFO - query_type :, vector


Created query engine with TreeSummarize synthesis
✅ Query engine with TreeSummarize created

🔍 Testing analytical query: 'Compare the advantages and disadvantages of different AI agent frameworks'


2026-02-08 02:51:16,555 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


📝 TreeSummarize Response:
Different AI agent frameworks offer various advantages and disadvantages based on their design and intended use cases.

**Advantages:**

1. **Autonomous Agents:**
   - **AutoGPT**: Known for its high complexity and ability to execute autonomous tasks, making it suitable for scenarios requiring minimal human intervention.
   - **BabyAGI**: Offers a simplified approach, which can be beneficial for those looking for an easier entry point into autonomous AI.

2. **Tool-Using Agents:**
   - **LangChain**: Provides a comprehensive framework for LLM applications, with a moderate learning curve, making it versatile for general applications.
   - **LlamaIndex**: Specializes in document understanding and retrieval-augmented generation (RAG), with a low complexity and easy learning curve, ideal for document Q&A tasks.

3. **Multi-Agent Systems:**
   - **CrewAI**: Facilitates team collaboration with a medium complexity and easy learning curve, making it effective for task

## 3. Structured Outputs with Pydantic Models

**Concept:** Structured outputs ensure predictable, parseable responses using Pydantic models. This is essential for API endpoints and data pipelines.

**Why it matters:** Instead of free-text responses, you get type-safe, validated data structures that applications can reliably process.

Complete the function below to create a structured output system for extracting research paper information.


In [16]:
# First, define the Pydantic models for structured outputs  
class ResearchPaperInfo(BaseModel):
    """Structured information about a research paper or AI concept."""
    title: str = Field(description="The main title or concept name")
    key_points: List[str] = Field(description="3-5 main points or findings")
    applications: List[str] = Field(description="Practical applications or use cases")
    summary: str = Field(description="Brief 2-3 sentence summary")

# Import the missing component
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.core.prompts import PromptTemplate

def create_structured_output_program(output_model: BaseModel = ResearchPaperInfo):
    """
    Create a structured output program using Pydantic models.
    
    TODO: Complete this function to create a structured output program.
    HINT: Use LLMTextCompletionProgram.from_defaults() with PydanticOutputParser and a prompt template
    
    Args:
        output_model: Pydantic model class for structured output
        
    Returns:
        LLMTextCompletionProgram that returns structured data
    """
    # 1. Create a prompt template that expects 'context' and 'query'
    # The program will use the Pydantic schema to instruct the LLM on the output format
    prompt_template_str = (
        "Context information is below.\n"
        "---------------------\n"
        "{context}\n"
        "---------------------\n"
        "Given the context information, please answer the following query in a structured format: {query}\n"
    )
    prompt = PromptTemplate(prompt_template_str)

    # 2. Create the structured output program
    # This automatically creates the PydanticOutputParser internally 
    # when you pass the output_cls
    program = LLMTextCompletionProgram.from_defaults(
        output_cls=output_model,
        prompt=prompt,
        llm=Settings.llm,
        verbose=True
    )

    # return program
    
    # PLACEHOLDER - Replace with actual implementation
    print(f"Created structured output program with {output_model.__name__}")
    return program

# Test the function
if index:
    structured_program = create_structured_output_program(ResearchPaperInfo)
    
    if structured_program:
        print("✅ Structured output program created")
        
        # Test with retrieval and structured extraction
        structure_query = "Tell me about AI agents and their capabilities"
        print(f"\n🔍 Testing structured query: '{structure_query}'")
        
        # Get context for structured extraction (Uncomment when implemented)
        retriever = VectorIndexRetriever(index=index, similarity_top_k=3)
        nodes = retriever.retrieve(structure_query)
        context = "\n".join([node.text for node in nodes])
        
        # Uncomment when implemented:
        response = structured_program(context=context, query=structure_query)
        print(f"📊 Structured Response:\n{response}")
        #print("   (Complete the function above to get structured JSON output)")
        
        print("\n💡 Expected output format:")
        print(f"   - title: {response.title}")
        print(f"   - key_points: {response.key_points}")
        print(f"   - applications: {response.applications}") 
        print(f"   - summary: {response.summary}")
    else:
        print("❌ Failed to create structured output program")
else:
    print("❌ No index available - run previous cells first")


2026-02-08 03:11:02,146 - INFO - query_type :, vector


Created structured output program with ResearchPaperInfo
✅ Structured output program created

🔍 Testing structured query: 'Tell me about AI agents and their capabilities'


2026-02-08 03:11:02,557 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


📊 Structured Response:
title='The Landscape of Emerging AI Agent Architectures for Reasoning, Planning, and Tool Calling: A Survey' key_points=['AI agents are capable of achieving complex goals through enhanced reasoning, planning, and tool execution.', 'Single-agent patterns perform well with defined personas, tools, and iterative feedback.', 'Multi-agent systems benefit from clear leadership, dynamic team composition, and intelligent communication filtering.', 'Current challenges include the lack of standardized benchmarks for evaluating AI agents and mitigating biases from underlying language models.', 'Future improvements are needed in real-world applicability and reliability of AI agents.'] applications=['Complex task execution requiring reasoning and planning', 'Dynamic team collaboration for problem-solving', 'Enhanced tool execution in AI-driven systems'] summary='This survey paper explores advancements in AI agent architectures, focusing on their capabilities in reasoning, pla

## 4. Advanced Pipeline - Combining All Techniques

**Concept:** Combine multiple advanced techniques into a single powerful query engine: similarity filtering + response synthesis + structured output.

**Why it matters:** Production RAG systems often need multiple techniques working together for optimal results.

Complete the function below to create a comprehensive advanced RAG pipeline.


In [17]:
def create_advanced_rag_pipeline(index, similarity_cutoff: float = 0.3, top_k: int = 10):
    """
    Create a comprehensive advanced RAG pipeline combining multiple techniques.
    
    TODO: Complete this function to create the ultimate advanced RAG query engine.
    HINT: Combine SimilarityPostprocessor + TreeSummarize using index.as_query_engine()
    
    Args:
        index: Vector index to query
        similarity_cutoff: Minimum similarity score for filtering
        top_k: Number of initial results to retrieve
        
    Returns:
        Advanced query engine with filtering and synthesis combined
    """
    # TODO: Create similarity postprocessor
    similarity_processor = SimilarityPostprocessor(similarity_cutoff=similarity_cutoff)
    
    # TODO: Create TreeSummarize for comprehensive responses
    tree_synthesizer = get_response_synthesizer(response_mode="tree_summarize", use_async=True)
    
    # TODO: Create the comprehensive query engine combining both techniques
    advanced_engine = index.as_query_engine(
        similarity_top_k=top_k,
        node_postprocessors=[similarity_processor],
        response_synthesizer=tree_synthesizer
    )
    
    # return advanced_engine
    
    # PLACEHOLDER - Replace with actual implementation
    print(f"Created advanced RAG pipeline with similarity_cutoff={similarity_cutoff}, top_k={top_k}")
    return advanced_engine

# Test the comprehensive pipeline
if index:
    advanced_pipeline = create_advanced_rag_pipeline(index)
    
    if advanced_pipeline:
        print("✅ Advanced RAG pipeline created successfully!")
        print("   🔧 Similarity filtering: ✅")
        print("   🌳 TreeSummarize synthesis: ✅")
        
        # Test with complex query
        complex_query = "Analyze the current state and future potential of AI agent technologies"
        print(f"\n🔍 Testing complex query: '{complex_query}'")
        
        # Uncomment when implemented:
        response = advanced_pipeline.query(complex_query)
        print(f"🚀 Advanced RAG Response:\n{response}")
        #print("   (Complete the function above to test the full pipeline)")
        
        print("\n🎯 This should provide:")
        print("   - Filtered relevant results only")
        print("   - Comprehensive analytical response")
        print("   - Combined postprocessing and synthesis")
    else:
        print("❌ Failed to create advanced RAG pipeline")
else:
    print("❌ No index available - run previous cells first")


2026-02-08 03:16:11,790 - INFO - query_type :, vector


Created advanced RAG pipeline with similarity_cutoff=0.3, top_k=10
✅ Advanced RAG pipeline created successfully!
   🔧 Similarity filtering: ✅
   🌳 TreeSummarize synthesis: ✅

🔍 Testing complex query: 'Analyze the current state and future potential of AI agent technologies'


2026-02-08 03:16:12,484 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-08 03:16:12,526 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-08 03:16:15,432 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


🚀 Advanced RAG Response:
AI agent technologies are currently in a promising state, with significant advancements in their ability to perform complex tasks such as reasoning, planning, and tool execution. Both single and multi-agent architectures have shown strong performance, especially when incorporating elements like defined personas, iterative feedback, and dynamic team structures. However, challenges remain, including the need for comprehensive benchmarks, real-world applicability, and bias mitigation.

In the financial services industry, AI agents are transforming operations by significantly boosting productivity and improving risk management. They have demonstrated productivity increases of 50-80% in financial data tasks and show promise in areas like algorithmic trading and fraud detection. Despite these advancements, challenges such as workforce upskilling, risk alignment, and regulatory compliance need to be addressed for successful deployment.

Looking to the future, AI agent

## 5. Final Test - Compare Basic vs Advanced RAG

Once you've completed all the functions above, run this cell to compare basic RAG with your advanced techniques.


In [18]:
# Final comparison: Basic vs Advanced RAG
print("🚀 Advanced RAG Techniques Assignment - Final Test")
print("=" * 60)

# Test queries for comparison
test_queries = [
    "What are the key capabilities of AI agents?",
    "How do you evaluate agent performance metrics?",
    "Explain the benefits and challenges of multimodal AI systems"
]

# Check if all components were created
components_status = {
    "Basic Index": index is not None,
    "Similarity Filter": 'filtered_engine' in locals() and filtered_engine is not None,
    "TreeSummarize": 'tree_engine' in locals() and tree_engine is not None,
    "Structured Output": 'structured_program' in locals() and structured_program is not None,
    "Advanced Pipeline": 'advanced_pipeline' in locals() and advanced_pipeline is not None
}

print("\n📊 Component Status:")
for component, status in components_status.items():
    status_icon = "✅" if status else "❌"
    print(f"   {status_icon} {component}")

# Create basic query engine for comparison
if index:
    print("\n🔍 Creating basic query engine for comparison...")
    basic_engine = index.as_query_engine(similarity_top_k=5)
    
    print("\n" + "=" * 60)
    print("🆚 COMPARISON: Basic vs Advanced RAG")
    print("=" * 60)
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n📋 Test Query {i}: '{query}'")
        print("-" * 50)
        
        # Basic RAG
        print("🔹 Basic RAG:")
        if basic_engine:
            # Uncomment when testing:
            basic_response = basic_engine.query(query)
            print(f"   Response: {str(basic_response)[:200]}...")
            print("   (Standard vector search + simple response)")
        
        # Advanced RAG (if implemented)
        print("\n🔸 Advanced RAG:")
        if components_status["Advanced Pipeline"]:
            # Uncomment when testing:
            advanced_response = advanced_pipeline.query(query)
            print(f"   Response: {advanced_response}")
            print("   (Filtered + TreeSummarize + Structured output)")
        else:
            print("   Complete the advanced pipeline function to test")

# Final status
print("\n" + "=" * 60)
print("🎯 Assignment Status:")
completed_count = sum(components_status.values())
total_count = len(components_status)

print(f"   Completed: {completed_count}/{total_count} components")

if completed_count == total_count:
    print("\n🎉 Congratulations! You've mastered Advanced RAG Techniques!")
    print("   ✅ Node postprocessors for result filtering")
    print("   ✅ Response synthesizers for better answers")
    print("   ✅ Structured outputs for reliable data")
    print("   ✅ Advanced pipelines combining all techniques")
    print("\n🚀 You're ready for production RAG systems!")
else:
    missing = total_count - completed_count
    print(f"\n📝 Complete {missing} more components to finish the assignment:")
    for component, status in components_status.items():
        if not status:
            print(f"   - {component}")

print("\n💡 Key learnings:")
print("   - Postprocessors improve result relevance and precision")
print("   - Different synthesizers work better for different query types")
print("   - Structured outputs enable reliable system integration")
print("   - Advanced techniques can be combined for production systems")


2026-02-08 03:18:18,381 - INFO - query_type :, vector


🚀 Advanced RAG Techniques Assignment - Final Test

📊 Component Status:
   ✅ Basic Index
   ✅ Similarity Filter
   ✅ TreeSummarize
   ✅ Structured Output
   ✅ Advanced Pipeline

🔍 Creating basic query engine for comparison...

🆚 COMPARISON: Basic vs Advanced RAG

📋 Test Query 1: 'What are the key capabilities of AI agents?'
--------------------------------------------------
🔹 Basic RAG:


2026-02-08 03:18:19,202 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-08 03:18:20,088 - INFO - query_type :, vector
2026-02-08 03:18:20,111 - INFO - Retrying request to /chat/completions in 0.447635 seconds


   Response: AI agents are designed to extend language model capabilities to solve real-world challenges. Their key capabilities include reasoning, planning, and the ability to call tools that interact with an ext...
   (Standard vector search + simple response)

🔸 Advanced RAG:


2026-02-08 03:18:20,729 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-08 03:18:21,109 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-08 03:18:22,984 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-08 03:18:23,840 - INFO - query_type :, vector


...

   Response: Agent performance metrics are evaluated using a variety of key performance indicators and benchmarks. These include measuring accuracy, which assesses the correctness of predictions and decisions, and efficiency, which looks at the speed and resource consumption in task completion. Robustness is another important metric, evaluating the system's ability to handle noisy or incomplete data and unexpected events. Specific benchmarks may focus on particular environments or tasks to determine how well agents can generalize to new situations. Additionally, human evaluation is often required for more nuanced or subjective measures, such as tool use efficiency and planning robustness. In multi-agent systems, performance is also assessed based on inter-agent communication effectiveness, collaborative plan execution, and task completion speed, with metrics like communication cost and time to task completion being commonly used.
   (Filtered + TreeSummarize + Structured output)

📋 Tes

2026-02-08 03:18:29,985 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-08 03:18:33,006 - INFO - query_type :, vector
2026-02-08 03:18:33,027 - INFO - Retrying request to /chat/completions in 0.477573 seconds


   Response: Multimodal AI systems offer several benefits, including the ability to process and integrate information from multiple sources such as text, images, and audio, which can lead to more comprehensive und...
   (Standard vector search + simple response)

🔸 Advanced RAG:


2026-02-08 03:18:33,809 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-08 03:18:34,171 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-08 03:18:36,805 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


   Response: Multimodal AI systems offer several benefits, such as enhanced understanding and decision-making capabilities by integrating information from various data types like text, images, and audio. This integration allows for more comprehensive analysis and insights, improving performance in complex tasks such as image captioning and sentiment analysis. Additionally, these systems can provide more natural and intuitive user interactions by leveraging the strengths of different modalities.

However, there are challenges associated with these systems. One major challenge is the complexity involved in effectively combining and synchronizing different data modalities, which often requires sophisticated algorithms and increased computational resources. Ensuring robustness and reliability across various modalities is also a concern, as errors in one modality can impact overall performance. Furthermore, developing and training these systems can be resource-intensive, requiring large dat